In [22]:
import pandas as pd

df_A_cleaned = pd.read_csv('../ground_truth_cleaned/D00XX_A_ground_truth_cleaned.csv', index_col=0)
df_B_cleaned = pd.read_csv('../ground_truth_cleaned/D00XX_B_ground_truth_cleaned.csv', index_col=0)
df_P_cleaned = pd.read_csv('../ground_truth_cleaned/P00XX_ground_truth_cleaned.csv', index_col=0)
df_A_cleaned['file'] = df_A_cleaned['file'].str.extract(r'/(D\d{4}_A)', expand=False)
df_B_cleaned['file'] = df_B_cleaned['file'].str.extract(r'/(D\d{4}_B)', expand=False)
df_P_cleaned['file'] = df_P_cleaned['file'].str.extract(r'/(P\d{4})', expand=False)
df_model = pd.read_csv(f'../predicted_transcription/Whisper_tiny.csv', index_col=0)
df_model = df_model.rename(columns={'Filename': 'file', 'Start_time': 'start_time', 'End_time': 'end_time'})
df_merged = pd.merge(df_A_cleaned, df_model, on=['file', 'start_time', 'end_time'], how='outer')
df_merged = pd.merge(df_B_cleaned, df_merged, on=['file', 'start_time', 'end_time'], how='right', suffixes=('_x',''))
df_merged['participant'] = df_merged['participant'].combine_first(df_merged['participant_x'])
df_merged['ground_truth'] = df_merged['ground_truth'].combine_first(df_merged['ground_truth_x'])
df_merged['ground_truth_cleaned'] = df_merged['ground_truth_cleaned'].combine_first(df_merged['ground_truth_cleaned_x'])
df_merged = df_merged.drop(columns=['participant_x', 'ground_truth_x', 'ground_truth_cleaned_x'])
df_merged = pd.merge(df_merged, df_P_cleaned, on=['file', 'start_time', 'end_time'], how='outer', suffixes=('','_x'))
df_merged['participant'] = df_merged['participant'].combine_first(df_merged['participant_x'])
df_merged['ground_truth'] = df_merged['ground_truth'].combine_first(df_merged['ground_truth_x'])
df_merged['ground_truth_cleaned'] = df_merged['ground_truth_cleaned'].combine_first(df_merged['ground_truth_cleaned_x'])
df_merged = df_merged.drop(columns=['participant_x', 'ground_truth_x', 'ground_truth_cleaned_x'])

df_A = df_merged[df_merged['file'].str.endswith('A')]
df_B = df_merged[df_merged['file'].str.startswith('P')]

sampled_A = df_A.sample(n=80, random_state=42)
sampled_B = df_B.sample(n=20, random_state=42)
sampled_df = pd.concat([sampled_A, sampled_B])

sampled_df.to_csv('sampled_hundred_lines.csv')